In [169]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import itertools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [220]:
f = np.arange(1, 89)
f = np.power(2, 1 / 12) ** (f - 49) * 440

note_to_index = dict(zip(['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B'], [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]))

def key_to_key_index(key_name: str):
    n = len(key_name)
    if n == 2:
        note = key_name[0]
        octave = key_name[1]
    else:
        assert n == 3
        note = key_name[:2]
        octave = key_name[2]
    octave = int(octave)
    note_index = note_to_index[note]
    key_index = (octave * 12) + note_index - 8
    return key_index
    
def key_to_frequency(key_name: str):
    key_index = key_to_key_index(key_name)
    frequency = f[key_index - 1]
    return frequency

def key_index_to_key(key_index: int):
    assert 1 <= key_index <= 88
    octave = (key_index + 8) // 12
    note_index = (key_index + 8) % 12
    note = itertools.islice(note_to_index, list(note_to_index.values()).index(note_index), None).__iter__().__next__()
    key_name = f'{note}{octave}'
    return key_name
    
assert key_to_key_index('C8') == key_to_key_index(key_index_to_key(key_to_key_index('C8')))

print(key_to_frequency('A0'))
print(key_to_frequency('A4'))
print(key_to_frequency('C8'))

def freq(key_name: str):
    return f[key_to_key_index(key_name)]

27.499999999999947
440.0
4186.009044809585


In [182]:
note_to_index = dict(zip(['C', 'C#', 'Db', 'D', 'D#', 'Eb', 'E', 'F', 'F#', 'Gb', 'G', 'G#', 'Ab', 'A', 'A#', 'Bb', 'B'], [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]))

def key_index_to_key(key_index: int):
    assert 1 <= key_index <= 88
    octave = (key_index + 8) // 12
    note_index = (key_index + 8) % 12
    note = next((k for k, v in note_to_index.items() if v == note_index), None)
#     note = itertools.islice(note_to_index, list(note_to_index.values()).index(note_index), None).__iter__().__next__()
    key_name = f'{note}{octave}'
    return key_name

assert key_to_key_index('C8') == key_to_key_index(key_index_to_key(key_to_key_index('C8')))

'A4'

In [130]:
g = lambda x, y: x / y
d = g(f[:, np.newaxis], f)
above = d[np.triu_indices(len(f), k=1)]
assert len(above) * 2 + 88 == 88 ** 2

plt.hist(above.ravel())
plt.xscale('log')
plt.show()
print(above.min())
print(above.max())
print(np.power(2, - 1 / 12))

In [141]:
just_intonation_vs_equal_temperament = {
    'Unison': [1 / 1, 1.0000, 1.0000],
    'Minor Second': [25 / 24, 1.0417, 1.05946],
    'Major Second': [9 / 8, 1.1250, 1.12246],
    'Minor Third': [6 / 5, 1.2000, 1.18921],
    'Major Third': [5 / 4, 1.2500, 1.25992],
    'Fourth': [4 / 3, 1.3333, 1.33483],
    'Diminished Fifth': [45 / 32, 1.4063, 1.41421],
    'Fifth': [3 / 2, 1.5000, 1.49831],
    'Minor Sixth': [8 / 5, 1.6000, 1.58740],
    'Major Sixth': [5 / 3, 1.6667, 1.68179],
    'Minor Seventh': [9 / 5, 1.8000, 1.78180],
    'Major Seventh': [15 / 8, 1.8750, 1.88775],
    'Octave': [2 / 1, 2.0000, 2.0000]
}

l = []
for k, v in just_intonation_vs_equal_temperament.items():
    a = v[0]
    b = v[2]
    l.append(abs(a - b))
detune = np.array(l)
print(sorted(detune))
threshold = 0.02

[0.0, 0.0, 0.0014966666666667017, 0.0016899999999999693, 0.0025399999999999867, 0.007959999999999967, 0.009919999999999929, 0.010789999999999855, 0.012600000000000167, 0.012750000000000039, 0.015123333333333155, 0.017793333333333328, 0.018199999999999994]


In [154]:
m = 2 ** 6

def n_to_ij(n):
    i = n // m
    j = n % m
    return i, j

def ij_to_n(i, j):
    return i * m + j

In [155]:
l = []
for i in range(m):
    for j in range(m):
        if i * j == 0:
            d = math.inf
        else:
            d = i / j
        l.append(d)
v = np.array(l)

In [161]:
def x_to_df(x):
    d = np.abs(v - x)
    ii, jj = zip(*[n_to_ij(n) for n in range(m * m)])
    df = pd.DataFrame({'i': ii, 'j': jj, 'd': d})
    return df

In [162]:
print(threshold)
print(f[23] / f[25])

df = x_to_df(f[23] / f[25])
df['sum'] = df['i'] + df['j']
df['prod'] = df['i'] * df['j']
df = df.loc[df['d'] < threshold]
df.sort_values(by=['d'])[:100]
print(df.loc[df['sum'] == df['sum'].min()])
print(df.loc[df['prod'] == df['prod'].min()])
print(df.nsmallest(5, 'sum'))
print(df.nsmallest(5, 'prod'))

0.02
0.8908987181403393
     i  j         d  sum  prod
456  7  8  0.015899   15    56
     i  j         d  sum  prod
456  7  8  0.015899   15    56
      i   j         d  sum  prod
456   7   8  0.015899   15    56
521   8   9  0.002010   17    72
586   9  10  0.009101   19    90
651  10  11  0.018192   21   110
912  14  16  0.015899   30   224
      i   j         d  sum  prod
456   7   8  0.015899   15    56
521   8   9  0.002010   17    72
586   9  10  0.009101   19    90
651  10  11  0.018192   21   110
912  14  16  0.015899   30   224


In [213]:
d = f[key_to_key_index('A#4')] / f[key_to_key_index('A4')]
print(d)
print(25 / 24)
print(14 / 13)
print(2 ** (1 / 12))
df = x_to_df(d)
df.loc[df['i'].isin([24, 25]) & df['j'].isin([24, 25]), :]
print(abs(2 ** (1 / 12) - 14 / 13))
print(abs(2 ** (1 / 12) - 25 / 24))

1.0594630943592953
1.0416666666666667
1.0769230769230769
1.0594630943592953
0.017459982563781562
0.01779642769262857


In [211]:
for i in np.arange(-12, 13):
    key_index = 52 + i
    print(key_index_to_key(key_index))
    d = f[key_index] / f[52]
    print(d)
    df = x_to_df(d)
    df['sum'] = df['i'] + df['j']
    df = df.loc[df['d'] < threshold]
#     df = df.loc[df['d'] < 0.1]
#     print(df.sort_values(by=['sum'])[:10])
    print(df.nsmallest(15, 'sum'))
    print()

C4
0.4999999999999998
      i   j             d  sum
66    1   2  2.220446e-16    3
132   2   4  2.220446e-16    6
198   3   6  2.220446e-16    9
264   4   8  2.220446e-16   12
330   5  10  2.220446e-16   15
396   6  12  2.220446e-16   18
462   7  14  2.220446e-16   21
528   8  16  2.220446e-16   24
594   9  18  2.220446e-16   27
660  10  20  2.220446e-16   30
726  11  22  2.220446e-16   33
792  12  24  2.220446e-16   36
793  12  25  2.000000e-02   37
858  13  26  2.220446e-16   39
859  13  27  1.851852e-02   40

C#4
0.5297315471796474
       i   j         d  sum
395    6  11  0.015723   17
461    7  13  0.008730   20
527    8  15  0.003602   23
593    9  17  0.000320   26
659   10  19  0.003416   29
725   11  21  0.005922   32
790   12  22  0.015723   34
791   12  23  0.007992   35
856   13  24  0.011935   37
857   13  25  0.009732   38
922   14  26  0.008730   40
923   14  27  0.011213   41
988   15  28  0.005983   43
989   15  29  0.012490   44
1054  16  30  0.003602   46

D4
0.5612

In [225]:
def get_candidate_ratios(d: float, top_n=None):
    df = x_to_df(d)
    df['sum'] = df['i'] + df['j']
    df = df.loc[df['d'] < threshold]
    if top_n is not None:
        return df.nsmallest(top_n, ['sum'])
    else:
        return df
    
d = freq('C#4') / freq('C4')
print(d)
get_candidate_ratios(d, top_n=1)

1.059463094359295


,i,j,d,sum
909,14,13,0.01746,27


In [245]:
for key_index in np.arange(key_to_key_index('A0'), key_to_key_index('C8')):
    fixed_key = 'C4'
    other_key = key_index_to_key(key_index)
    f0 = freq(fixed_key)
    f1 = freq(other_key)
    d = f0 / f1
    row = get_candidate_ratios(d, top_n=1)
    print(f'{fixed_key}'.rjust(3), ' + ', f'{other_key}'.rjust(3), ': ', end='', sep='')
    if len(row) == 0:
        print('None')
    else:
        r = row['d'].item()
        i = row['i'].item()
        j = row['j'].item()
        print(f'{i}'.rjust(2), f' {j}'.rjust(3), f' {r:0.3f}', sep='')

 C4 +  A0: 19  2 0.014
 C4 + A#0: None
 C4 +  B0: None
 C4 +  C1:  8  1 0.000
 C4 + C#1: None
 C4 +  D1: 50  7 0.016
 C4 + D#1: 47  7 0.013
 C4 +  E1: 19  3 0.016
 C4 +  F1:  6  1 0.007
 C4 + F#1: 17  3 0.010
 C4 +  G1: 16  3 0.006
 C4 + G#1: None
 C4 +  A1: 19  4 0.007
 C4 + A#1:  9  2 0.010
 C4 +  B1: 17  4 0.012
 C4 +  C2:  4  1 0.000
 C4 + C#2: 34  9 0.002
 C4 +  D2: 25  7 0.008
 C4 + D#2: 27  8 0.011
 C4 +  E2: 19  6 0.008
 C4 +  F2:  3  1 0.003
 C4 + F#2: 17  6 0.005
 C4 +  G2:  8  3 0.003
 C4 + G#2:  5  2 0.020
 C4 +  A2: 19  8 0.003
 C4 + A#2:  9  4 0.005
 C4 +  B2: 17  8 0.006
 C4 +  C3:  2  1 0.000
 C4 + C#3: 15  8 0.013
 C4 +  D3:  9  5 0.018
 C4 + D#3:  5  3 0.015
 C4 +  E3:  8  5 0.013
 C4 +  F3:  3  2 0.002
 C4 + F#3:  7  5 0.014
 C4 +  G3:  4  3 0.002
 C4 + G#3:  5  4 0.010
 C4 +  A3:  6  5 0.011
 C4 + A#3:  9  8 0.003
 C4 +  B3: 14 13 0.017
 C4 +  C4:  1  1 0.000
 C4 + C#4: 13 14 0.015
 C4 +  D4:  7  8 0.016
 C4 + D#4:  5  6 0.008
 C4 +  E4:  4  5 0.006
 C4 +  F4:  3  4